# 랜덤포레스트 분류

## #01. 패키지

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt
import seaborn as sb
# 회귀를 위한 더미변수 생성
from pandas import read_excel, DataFrame, melt

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score, classification_report

# 데이터 불균형 해소를 위한 smpling 패키지
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

## #02. 데이터

| 필드명 | 설명  |
|---|---|
| id | 고유 식별 번호 |
| age | 나이 |
| height(cm) | 키 |
| weight(kg) | 몸무게 |
| waist(cm) | 허리둘레 |
| eyesight(left) | 시력(왼쪽) |
| eyesight(right) | 시력(오른쪽) |
| hearing(left) | 청력(왼쪽) |
| hearing(right) | 청력(오른쪽) |
| systolic | 수축기 혈압(mmHg 단위) |
| relaxation | 휴식 혈압(mmHg 단위) |
| fasting blood sugar | 공복 혈당 수치(mg/dL 단위) |
| Cholesterol | 콜레스테롤 수치(mg/dL 단위) |
| triglyceride | 중성지방 수치(mg/dL 단위) |
| HDL | 고밀도 지단백 수치 (mg/dL) |
| LDL | 저밀도 지단백 수치 (mg/dL) |
| hemoglobin | 헤모글로빈 수치(g/dL) |
| Urine protein | 소변내 단백질 수준 |
| serum creatinine | 혈청 크레아티닌 수치(mg/dL) |
| AST | 아스파르트 아미노전이효소(AST) 수준 |
| ALT | 알라닌아미노 전이효소 수준 |
| Gtp | 감마-글루타밀 전이효소 수준 |
| dental caries | 1인당 치아우식증 유무를 나타내는 값(0: 없음, 1: 있음) |
| smoking | 흡연상태(0: 비흡연자, 1: 흡연자) |

In [ ]:
origin = read_excel("https://data.hossam.kr/G02/smoker_status.xlsx")
# print(origin.info())
origin.head().T

### 데이터 확인

In [ ]:
print(origin.shape)

In [ ]:
origin.isnull().any()

In [ ]:
origin.info()

### 목적변수 비율 확인

In [ ]:
origin.columns

In [ ]:
plt.figure(figsize=(5,5))
plt.pie(origin['smoking'].value_counts(), labels=['non-smoker', 'smoker'], autopct='%1.1f%%')
plt.show()
plt.close()

### 변수 값 분리(종속/독립변수)

In [ ]:
x = origin.drop('smoking', axis=1)
y = origin['smoking']
x.shape, y.shape

### 훈련/검증데이터 분할

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=777)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

### 데이터 불균형 해소

#### Over Sampling - SMOTE

소수 집단의 데이터를 바탕으로 새로운 데이터를 생성.

단순히 소수 집단의 데이터를 복원 추출하는 것이 아니라 소수 집단 데이터를 분석해 어떤 특징이 있는지 살피고 그와 유사한 패턴을 가지는 가짜 데이터를 생성.

##### `sampling_strategy 파라미터`

sampling_strategy : 2진 분류일 경우 실수로 설정 가능

| 값 | 설명 |
| -- | -- |
| `minority` | `소수 클래스만` 다시 샘플링 |
| `not majority` | `다수 아님` : 다수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `not minority` | `소수 아님` : 소수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `all` | `모든 클래스`를 다시 샘플링 |
| `auto` | 자동 처리 |

혹은 실수 타입으로 설정할 경우 샘플 수의 비율을 의미

##### `k_neighbors 파라미터 (int)`

합성 샘플을 생성하는데 사용할 샘플의 가장 가까운 이웃 수 (기본값=5)

In [ ]:
smote_sampler = SMOTE(sampling_strategy='minority', random_state=777)
x_sm, y_sm = smote_sampler.fit_resample(x_train, y_train)
print(x_sm.shape, y_sm.shape)

y_sm.value_counts().sort_index()

## #04. 훈련모델 적합

### RandomForestClassifier 하이퍼 파라미터

| n_estimators | 결정 트리의 갯수를 지정(기본값=10). 즉, 반복 횟수.<br/>성능에 비례, 속도에 반비례 |
| -- | -- |
| min_samples_split | 노드를 분할하기 위한 최소한의 샘플 데이터 수.<br/>과적합을 제어하는 데 사용(기본값=2), 값이 작을 수록 분할 노드가 증가하여 과적합 가능성이 높아짐. |
| min_samples_leaf | 리프노드가 되기 위한 최소한의 샘플 데이터 수.<br/>과적합을 제어하는데 사용 |
| max_features | 최적의 분할을 위해 고려할 최대 feature 개수(기본값=auto), int 형일 경우 갯수, float 형일 경우 비율 |
| max_depth | 트리의 최대 깊이(기본값=None).<br/>`max_depth`가 `None`일 경우 완벽하게 클래스 값이 결정되거나 데이터 개수가 min_samples_split에서 설정한 값보다 작아질 때 까지 분할 |
| max_leaf_nodes | 리프 노드의 최대 개수 |

### 하이퍼 파라미터 튜닝

> GridSearchCV

`cv` : 쪼개는 단위

`n_jobs` : 실행할 병렬 작업의 수. CPU의 프로세스 수만큼 설정 가능. -1은 모든 프로세서를 사용함을 의미.

In [ ]:
# 모델 생성
rfc = RandomForestClassifier(random_state=777)

# 사용할 파라미터 설정
params = {
    'random_state':[20,50,100],
    "max_depth":[5,30,100],
    # 아래거 실행하면 시간 오래 걸림
    # 'min_samples_split':[2,5,10],
    # 'min_samples_leaf':[1,5,10]
}

# 모델 생성
grid = GridSearchCV(rfc, param_grid=params, cv=5, n_jobs=-1)
# 학습
grid.fit(x_sm, y_sm)

print("최적의 하이퍼 파라미터 :",grid.best_params_)
print("최적의 모델 평균 성능(훈련데이터) :",grid.best_score_)

# 검증 정확도 유도
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)
print("최종 모델의 성능(테스트 데이터) :", accuracy_score(y_test,y_pred))

### 분류 보고서

- precision : 정밀도(양성 클래스라고 예측한 샘플 중 실제로 양성 클래스에 속하는 샘플). 높을 수록 좋음
- recall : 재현율, 실제로 양성 클래스에 속한 샘플 중에서 양성 클래스라고 예측한 샘플 수의 비율. 높을 수록 좋음
- f1-score : 정밀도와 재현율의 가중 조화 평균값. 높을 수록 좋음
- support : 각 Label에 대한 실제 샘플 수. 높을 수록 좋음
- accuracy : 정확도. 전체 샘플 중 맞게 예측한 샘플 수의 비율. 높을 수록 좋음
- macro avg : 단순 평균값(샘플 수의 불균형을 고려하지 않은 값)
- weighted avg : 각 클래스에 속하는 표본의 개수로 가중 평균을 낸 값(샘플 수의 불균형을 고려한 값)

In [ ]:
print(classification_report(y_test, y_pred))